# Reinforcement Learning para Trading - Explarando o yFinance.

## Objetivo
Entendimento de como usar o modulo yfinance e importar dados do tickers para exploração do mercado financeiro.


In [67]:
import yfinance as yf
import pandas as pd



In [68]:
#DFPETR4 = yf.Ticker("PETR4.SA")
#DFPETR3 = yf.Ticker("PETR3.SA")
#DFPETR3.history(period='1mo')

tickersymbols = ["PETR3.SA", "PETR4.SA", "VALE3.SA", "BRFS3.SA"]
tickers = yf.Tickers(" ".join(tickersymbols))

for symbol in tickersymbols:
    info = tickers.tickers[symbol].info
    print(f"Ticker: {symbol}")
    print(f"Name: {info.get('longName', 'N/A')}")
    print(f"Sector: {info.get('sector', 'N/A')}")
    print(f"Market Cap: {info.get('marketCap', 'N/A')}")

Ticker: PETR3.SA
Name: Petróleo Brasileiro S.A. - Petrobras
Sector: Energy
Market Cap: 398814478336
Ticker: PETR4.SA
Name: Petróleo Brasileiro S.A. - Petrobras
Sector: Energy
Market Cap: 398814445568
Ticker: PETR4.SA
Name: Petróleo Brasileiro S.A. - Petrobras
Sector: Energy
Market Cap: 398814445568
Ticker: VALE3.SA
Name: Vale S.A.
Sector: Basic Materials
Market Cap: 263511703552
Ticker: VALE3.SA
Name: Vale S.A.
Sector: Basic Materials
Market Cap: 263511703552
Ticker: BRFS3.SA
Name: BRF S.A.
Sector: Consumer Defensive
Market Cap: 28579856384
Ticker: BRFS3.SA
Name: BRF S.A.
Sector: Consumer Defensive
Market Cap: 28579856384


In [69]:
# Get historical data for PETR3.SA
DFPETR3 = pd.DataFrame(tickers.tickers[tickersymbols[0]].history(period='1y'))

# Get historical data for PETR4.SA
DFPETR4 = pd.DataFrame(tickers.tickers[tickersymbols[1]].history(period='1y'))  

# Get historical data for VALE3.SA
DFVALE3 = pd.DataFrame(tickers.tickers[tickersymbols[2]].history(period='1y'))

# Get historical data for BRFS3.SA
DFBRFS3 = pd.DataFrame(tickers.tickers[tickersymbols[3]].history(period='1y'))

In [70]:
DFPETR3.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-10-24 00:00:00-03:00,33.834328,33.938433,33.591416,33.843002,7272300,0.0,0.0
2024-10-25 00:00:00-03:00,33.843001,34.372205,33.825653,34.181347,7829100,0.0,0.0
2024-10-28 00:00:00-03:00,33.556713,34.163996,33.374529,34.111942,5792000,0.0,0.0
2024-10-29 00:00:00-03:00,34.285456,34.354858,33.929762,34.025192,4782500,0.0,0.0
2024-10-30 00:00:00-03:00,34.103266,34.337504,33.704192,33.730221,7560600,0.0,0.0


In [71]:
DFPETR3.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 250 entries, 2024-10-24 00:00:00-03:00 to 2025-10-24 00:00:00-03:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          250 non-null    float64
 1   High          250 non-null    float64
 2   Low           250 non-null    float64
 3   Close         250 non-null    float64
 4   Volume        250 non-null    int64  
 5   Dividends     250 non-null    float64
 6   Stock Splits  250 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 15.6 KB


# - Reinforcement Learning para Trading PETR3

## Estratégia: Metodológica e Estruturada (Estilo Original)

### Introdução
- **Preparação meticulosa**: Base teórica sólida seguindo padrões das aulas
- **Abordagem conservadora**: Implementação tradicional e confiável
- **Metodologia comprovada**: Código limpo e bem estruturado
- **Foco pedagógico**: Prioriza entendimento dos fundamentos

### Objetivo
Criar um agente de RL para automatizar decisões de trading da Petrobras (PETR3) com base em dados históricos.

### Componentes do Sistema RL:
1. **Preparação dos Dados** - Usar preço de fechamento
2. **Definição do Estado** - Preço atual + janela histórica
3. **Definição das Ações** - Comprar (1), Vender (-1), Hold (0)
4. **Ambiente Básico** - Simulação simples com capital inicial

## PASSO 1: Preparação dos Dados
Vamos preparar os dados do PETR3 para o ambiente de RL, focando no preço de fechamento.

In [72]:
# PASSO 1: Preparação dos Dados - Usando apenas preço de fechamento
import numpy as np

# Extrair apenas os preços de fechamento
prices = DFPETR3['Close'].values
print(f"Dados de preços: {len(prices)} dias")
print(f"Primeiro preço: R$ {prices[0]:.2f}")
print(f"Último preço: R$ {prices[-1]:.2f}")
print(f"Preço mínimo: R$ {prices.min():.2f}")
print(f"Preço máximo: R$ {prices.max():.2f}")

Dados de preços: 250 dias
Primeiro preço: R$ 33.84
Último preço: R$ 31.75
Preço mínimo: R$ 30.26
Preço máximo: R$ 39.56


## PASSO 2: Definição do Estado (State Space)
O estado será uma janela dos últimos preços para que o agente possa "ver" o histórico recente.

In [73]:
# PASSO 2: Definição do Estado - Janela de preços históricos
WINDOW_SIZE = 5  # Janela de 5 dias para simplificar

def create_state(prices, current_index, window_size=WINDOW_SIZE):
    """Cria o estado atual baseado em uma janela de preços históricos"""
    if current_index < window_size:
        # Para os primeiros dias, preencher com o primeiro preço
        state = np.full(window_size, prices[0])
        state[-current_index-1:] = prices[:current_index+1]
    else:
        # Janela normal de preços
        state = prices[current_index-window_size+1:current_index+1]
    
    # Normalizar os preços (dividir pelo preço atual)
    current_price = prices[current_index]
    normalized_state = state / current_price
    
    return normalized_state

# Teste do estado
test_index = 10
test_state = create_state(prices, test_index)
print(f"Estado no dia {test_index}: {test_state}")
print(f"Preço atual (dia {test_index}): R$ {prices[test_index]:.2f}")

Estado no dia 10: [0.99791556 0.99895766 0.99400719 0.99296509 1.        ]
Preço atual (dia 10): R$ 33.30


## PASSO 3: Definição das Ações (Action Space)
O agente pode tomar 3 ações básicas: Comprar, Vender ou Manter posição.

In [74]:
# PASSO 3: Definição das Ações
# 0 = Hold (manter posição)
# 1 = Buy (comprar)  
# 2 = Sell (vender)

ACTION_HOLD = 0
ACTION_BUY = 1  
ACTION_SELL = 2

# Mapeamento para interpretação
ACTION_NAMES = {
    0: "HOLD",
    1: "BUY", 
    2: "SELL"
}

print("Ações disponíveis para o agente:")
for action, name in ACTION_NAMES.items():
    print(f"  {action}: {name}")
    
# Função para converter ação em texto
def action_to_text(action):
    return ACTION_NAMES.get(action, "UNKNOWN")

Ações disponíveis para o agente:
  0: HOLD
  1: BUY
  2: SELL


## PASSO 4: Ambiente Básico de Trading
Vamos criar um ambiente simples que simula um portfólio com capital inicial.

In [75]:
# PASSO 4: Ambiente Básico de Trading
class SimpleTradingEnvironment:
    def __init__(self, prices, initial_capital=10000.0):
        self.prices = prices
        self.initial_capital = initial_capital
        self.reset()
    
    def reset(self):
        """Reinicia o ambiente"""
        self.current_step = WINDOW_SIZE  # Começar após a janela inicial
        self.cash = self.initial_capital
        self.shares = 0
        self.portfolio_values = []
        return self.get_state()
    
    def get_state(self):
        """Retorna o estado atual"""
        return create_state(self.prices, self.current_step)
    
    def get_portfolio_value(self):
        """Calcula o valor total do portfólio"""
        current_price = self.prices[self.current_step]
        return self.cash + (self.shares * current_price)
    
    def step(self, action):
        """Executa uma ação e retorna (next_state, reward, done, info)"""
        current_price = self.prices[self.current_step]
        
        # Executar a ação
        if action == ACTION_BUY and self.cash >= current_price:
            # Comprar uma ação
            self.shares += 1
            self.cash -= current_price
            
        elif action == ACTION_SELL and self.shares > 0:
            # Vender uma ação
            self.shares -= 1
            self.cash += current_price
        
        # (ACTION_HOLD não faz nada)
        
        # Calcular recompensa (mudança no valor do portfólio)
        portfolio_value = self.get_portfolio_value()
        self.portfolio_values.append(portfolio_value)
        
        if len(self.portfolio_values) > 1:
            reward = portfolio_value - self.portfolio_values[-2]
        else:
            reward = 0
        
        # Avançar para o próximo dia
        self.current_step += 1
        done = self.current_step >= len(self.prices) - 1
        
        next_state = self.get_state() if not done else None
        
        info = {
            'cash': self.cash,
            'shares': self.shares,
            'portfolio_value': portfolio_value,
            'current_price': current_price
        }
        
        return next_state, reward, done, info

# Criar o ambiente
env = SimpleTradingEnvironment(prices)
print("Ambiente criado com sucesso!")
print(f"Capital inicial: R$ {env.initial_capital:,.2f}")
print(f"Preços disponíveis: {len(prices)} dias")

Ambiente criado com sucesso!
Capital inicial: R$ 10,000.00
Preços disponíveis: 250 dias


## Teste do Ambiente
Vamos testar o ambiente com algumas ações manuais para verificar se está funcionando.

In [76]:
# Teste do Ambiente - Simulação de algumas ações
env_test = SimpleTradingEnvironment(prices)
state = env_test.reset()

print("=== TESTE DO AMBIENTE ===")
print(f"Estado inicial: {state}")
print(f"Cash inicial: R$ {env_test.cash:,.2f}")
print(f"Ações iniciais: {env_test.shares}")
print(f"Valor portfólio inicial: R$ {env_test.get_portfolio_value():,.2f}")
print()

# Simular 5 ações
test_actions = [ACTION_BUY, ACTION_BUY, ACTION_HOLD, ACTION_SELL, ACTION_HOLD]

for i, action in enumerate(test_actions):
    print(f"--- Passo {i+1}: {action_to_text(action)} ---")
    next_state, reward, done, info = env_test.step(action)
    
    print(f"Reward: R$ {reward:+.2f}")
    print(f"Cash: R$ {info['cash']:,.2f}")
    print(f"Ações: {info['shares']}")
    print(f"Preço atual: R$ {info['current_price']:.2f}")
    print(f"Valor portfólio: R$ {info['portfolio_value']:,.2f}")
    print()
    
    if done:
        print("Simulação terminada!")
        break

print("=== TESTE CONCLUÍDO ===")
print(f"Capital inicial: R$ {env_test.initial_capital:,.2f}")
print(f"Valor final: R$ {env_test.get_portfolio_value():,.2f}")
print(f"Lucro/Prejuízo: R$ {env_test.get_portfolio_value() - env_test.initial_capital:+,.2f}")

=== TESTE DO AMBIENTE ===
Estado inicial: [1.00870449 1.00665634 1.00409631 0.99539159 1.        ]
Cash inicial: R$ 10,000.00
Ações iniciais: 0
Valor portfólio inicial: R$ 10,000.00

--- Passo 1: BUY ---
Reward: R$ +0.00
Cash: R$ 9,966.11
Ações: 1
Preço atual: R$ 33.89
Valor portfólio: R$ 10,000.00

--- Passo 2: BUY ---
Reward: R$ -0.66
Cash: R$ 9,932.89
Ações: 2
Preço atual: R$ 33.23
Valor portfólio: R$ 9,999.34

--- Passo 3: HOLD ---
Reward: R$ +0.07
Cash: R$ 9,932.89
Ações: 2
Preço atual: R$ 33.26
Valor portfólio: R$ 9,999.41

--- Passo 4: SELL ---
Reward: R$ -0.33
Cash: R$ 9,965.98
Ações: 1
Preço atual: R$ 33.10
Valor portfólio: R$ 9,999.08

--- Passo 5: HOLD ---
Reward: R$ -0.03
Cash: R$ 9,965.98
Ações: 1
Preço atual: R$ 33.06
Valor portfólio: R$ 9,999.05

=== TESTE CONCLUÍDO ===
Capital inicial: R$ 10,000.00
Valor final: R$ 9,999.28
Lucro/Prejuízo: R$ -0.72
